In [1]:
! pip install faiss-cpu python-dotenv telebot nltk paddleocr paddlepaddle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.0/195.0 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
!pip install langchain langchain_community

In [3]:
from langchain_community.llms import Ollama

In [4]:
!pip install -U datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [5]:
from datasets import load_dataset
from langchain.docstore.document import Document

# Load dataset
ds = load_dataset("lewtun/drug-reviews")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.jsonl:   0%|          | 0.00/97.7M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/32.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/161297 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/53766 [00:00<?, ? examples/s]

In [6]:
train_data = ds["train"].filter(lambda x: x['review'] is not None and x['condition'] is not None)

seen_drugs = set()
documents = []

for example in train_data:
    drug = example['drugName']
    if drug not in seen_drugs:
        condition = example['condition']
        review = example['review']
        rating = example['rating']
        text = f"Drug: {drug}\nCondition: {condition}\nRating: {rating}/10\nReview: {review}"
        documents.append(Document(page_content=text))
        seen_drugs.add(drug)

print(f"Collected {len(documents)} unique drugs.")

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Collected 3431 unique drugs.


In [7]:
len(seen_drugs)

3431

In [8]:
len(documents)

3431

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(documents)
print(f"Medical dataset split into {len(chunks)} chunks.")

Medical dataset split into 3443 chunks.


In [11]:
chunks

[Document(metadata={}, page_content='Drug: Valsartan\nCondition: Left Ventricular Dysfunction\nRating: 9.0/10\nReview: "It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"'),
 Document(metadata={}, page_content='Drug: Guanfacine\nCondition: ADHD\nRating: 8.0/10\nReview: "My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medication

In [12]:
# !curl -fsSL https://ollama.com/install.sh | sh

# !ollama serve &

# !ollama pull deepseek-r1:1.5b

# !ollama run deepseek-r1:1.5b

In [13]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [14]:
!pip install colab-xterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 3.7 MB/s eta 0:00:00


In [49]:
%reload_ext colabxterm
%xterm

Launching Xterm...

<IPython.core.display.Javascript object>

In [19]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=60169bf9773f0dc1e047c1c2b291d3da6255bf653fb881a0a4fff479bde7677e
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [1]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
from IPython.display import Markdown
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langdetect import detect
import subprocess


ModuleNotFoundError: No module named 'langchain_community'

In [21]:
# llm = ChatOllama(model="deepseek-r1:8b")
llm = ChatOllama(model="deepseek-r1:1.5b")

/tmp/ipython-input-21-2279837865.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="deepseek-r1:1.5b")


In [22]:
# Arabic to English
ar_en_tokenizer = AutoTokenizer.from_pretrained("Abdalrahmankamel/NLLB_Egyptian_Arabic_to_English")
ar_en_model = AutoModelForSeq2SeqLM.from_pretrained("Abdalrahmankamel/NLLB_Egyptian_Arabic_to_English")

# English to Arabic
en_ar_tokenizer = AutoTokenizer.from_pretrained("NAMAA-Space/masrawy-english-to-egyptian-arabic-translator-v2.9")
en_ar_model = AutoModelForSeq2SeqLM.from_pretrained("NAMAA-Space/masrawy-english-to-egyptian-arabic-translator-v2.9")

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/957M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [23]:
def translate_ar_to_en(text):
    inputs = ar_en_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = ar_en_model.generate(**inputs)
    return ar_en_tokenizer.decode(outputs[0], skip_special_tokens=True)

def translate_en_to_ar(text):
    inputs = en_ar_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = en_ar_model.generate(**inputs)
    return en_ar_tokenizer.decode(outputs[0], skip_special_tokens=True)


In [24]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def get_relevant_chunks(question, k=5):
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    chunk_embeddings = embedder.encode([c.page_content for c in chunks], convert_to_tensor=True)

    scores = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    top_k_idx = scores.argsort(descending=True)[:k]

    return [chunks[i] for i in top_k_idx]

prompt = ChatPromptTemplate.from_messages([
    ("system",
    """You are a highly qualified and empathetic medical doctor specializing in pharmacology and diagnostics.
Your task is to assist patients by analyzing symptoms and suggesting possible over-the-counter or prescription treatments,
using only the provided medical context and patient history.

Please follow these guidelines strictly:
1. Summarize or rephrase patient reviews professionally — focus on effectiveness, side effects, and usage patterns.
2. Only recommend medications based on the drug context. Do NOT invent drugs or diagnoses not found in the context.
3. If reviews indicate serious side effects, mention them clearly as warnings.
4. If symptoms are severe, radiating, or persistent, recommend immediate consultation with a licensed healthcare provider.
5. If the patient asks about **dosage, price, amount, or frequency**, respond clearly that they must refer to a licensed doctor or read the official instructions — do not make assumptions or guesses.
6. Write in a professional tone, like a medical consultation summary. Avoid overly casual language.
7.if the question is in arabic, answer with arabic only

Return your answer as:
- **Patient Query:** Rephrased version of the user’s question
- **Doctor's Recommendations:** Suggested treatments based on the above
- **Safety Warnings (if any):** Explicit cautions based on the reviews or symptoms
"""),

    MessagesPlaceholder(variable_name="chat_history"),

    ("human", "Patient: {question}\n\nMedical Records:\n{context}")
])

prompt_with_context = prompt.partial()
chain = create_stuff_documents_chain(llm, prompt)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def ask_question_with_memory(question, k=5):
    relevant_chunks = get_relevant_chunks(question, k)
    context = "\n\n".join([chunk.page_content for chunk in relevant_chunks])

    memory.chat_memory.add_user_message(question)

    prompt_with_context = prompt.partial(context=context)
    chain = create_stuff_documents_chain(llm, prompt_with_context)

    result = chain.invoke({
        "question": question,
        "chat_history": memory.chat_memory.messages
    })

    memory.chat_memory.add_ai_message(result)
    return result

def get_relevant_chunks(question, k=5):
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    chunk_embeddings = embedder.encode([c.page_content for c in chunks], convert_to_tensor=True)
    scores = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    top_k_idx = scores.argsort(descending=True)[:k]
    return [chunks[i] for i in top_k_idx]

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = create_stuff_documents_chain(llm, prompt)

import re

def clean_response(response_text):
    return re.sub(r"<think>.*?</think>", "", response_text, flags=re.DOTALL).strip()

def ask_question_with_memory(question, k=5):
    relevant_chunks = get_relevant_chunks(question, k)
    context_docs = [Document(page_content=chunk.page_content) for chunk in relevant_chunks]

    memory.chat_memory.add_user_message(question)
    chain = create_stuff_documents_chain(llm, prompt)

    result = chain.invoke({
        "question": question,
        "context": context_docs,
        "chat_history": memory.chat_memory.messages
    })

    cleaned = clean_response(result)
    memory.chat_memory.add_ai_message(cleaned)
    return cleaned

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-24-808056067.py:43: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def get_relevant_chunks(question, k=5):
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    chunk_embeddings = embedder.encode([c.page_content for c in chunks], convert_to_tensor=True)

    scores = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    top_k_idx = scores.argsort(descending=True)[:k]

    return [chunks[i] for i in top_k_idx]

prompt = ChatPromptTemplate.from_messages([
    ("system",
    """You are a highly qualified and empathetic medical doctor specializing in pharmacology and diagnostics.
Your task is to assist patients by analyzing symptoms and suggesting possible over-the-counter or prescription treatments,
using only the provided medical context and patient history.

Please follow these guidelines strictly:
1. Summarize or rephrase patient reviews professionally — focus on effectiveness, side effects, and usage patterns.
2. Only recommend medications based on the drug context. Do NOT invent drugs or diagnoses not found in the context.
3. If reviews indicate serious side effects, mention them clearly as warnings.
4. If symptoms are severe, radiating, or persistent, recommend immediate consultation with a licensed healthcare provider.
5. If the patient asks about **dosage, price, amount, or frequency**, respond clearly that they must refer to a licensed doctor or read the official instructions — do not make assumptions or guesses.
6. Write in a professional tone, like a medical consultation summary. Avoid overly casual language.
7.if the question is in arabic, answer with arabic only

Return your answer as:
- **Patient Query:** Rephrased version of the user’s question
- **Doctor's Recommendations:** Suggested treatments based on the above
- **Safety Warnings (if any):** Explicit cautions based on the reviews or symptoms
"""),

    MessagesPlaceholder(variable_name="chat_history"),

    ("human", "Patient: {question}\n\nMedical Records:\n{context}")
])

prompt_with_context = prompt.partial()
chain = create_stuff_documents_chain(llm, prompt)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def ask_question_with_memory(question, k=5):
    relevant_chunks = get_relevant_chunks(question, k)
    context = "\n\n".join([chunk.page_content for chunk in relevant_chunks])

    memory.chat_memory.add_user_message(question)

    prompt_with_context = prompt.partial(context=context)
    chain = create_stuff_documents_chain(llm, prompt_with_context)

    result = chain.invoke({
        "question": question,
        "chat_history": memory.chat_memory.messages
    })

    memory.chat_memory.add_ai_message(result)
    return result

def get_relevant_chunks(question, k=5):
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    chunk_embeddings = embedder.encode([c.page_content for c in chunks], convert_to_tensor=True)
    scores = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    top_k_idx = scores.argsort(descending=True)[:k]
    return [chunks[i] for i in top_k_idx]

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = create_stuff_documents_chain(llm, prompt)

import re

def clean_response(response_text):
    return re.sub(r"<think>.*?</think>", "", response_text, flags=re.DOTALL).strip()

def ask_question_with_memory(question, k=5):
    relevant_chunks = get_relevant_chunks(question, k)
    context_docs = [Document(page_content=chunk.page_content) for chunk in relevant_chunks]

    memory.chat_memory.add_user_message(question)
    chain = create_stuff_documents_chain(llm, prompt)

    result = chain.invoke({
        "question": question,
        "context": context_docs,
        "chat_history": memory.chat_memory.messages
    })

    cleaned = clean_response(result)
    memory.chat_memory.add_ai_message(cleaned)
    return cleaned

In [26]:
def detect_drug(query):
    query_lower = query.lower()
    for drug in seen_drugs:
        if drug.lower() in query_lower:
            return drug
    return None


In [27]:
def query_ollama_rag(query):
    detected_drug = detect_drug(query)

    if detected_drug:
        print(f"Detected drug in dataset: {detected_drug}")
        return ask_question_with_memory(query)
        # context = "\n\n".join([f"Review {i+1}: {r}" for i, r in enumerate(reviews)])
        # answer = chain.run(context=context, question=query)
        # print("Ollama Answer from Dataset:\n", answer)
    else:
        print("Drug not found in dataset. Falling back to internet-based response...\n")
        fallback_prompt = f"Use your internet knowledge to answer this medical question:\n\n{query}\n\nAnswer:"
        result = subprocess.run(["ollama", "run", "gemma2:2b"], input=fallback_prompt.encode(), capture_output=True)
        print("Ollama Answer from Internet:\n", result.stdout.decode())


In [36]:
question = "what is the Bupropion medicine for ?"
response = ask_question_with_memory(question,k=3)
Markdown(response)

Detected drug in dataset: Bupropion


**Patient Query:** What is the Bupropion medicine for?

**Doctor's Recommendations:**
- **Main Use:**  
  Bupropion is primarily used to alleviate ulcers, mouth sores, and postmenstrual pain. It helps reduce inflammation in these conditions.  
  For weight loss or other conditions, naltrexone may be added as part of a combined medication regimen.

- **Specific Conditions and Side Effects:**  
  - **For smoking cessation:**  
    Bupropion is recommended for its ease of use and ability to prevent ulcers or sores caused by smoking. It also helps with postmenstrual pain.  
    - Note: The user should consult a healthcare provider before taking it, as it may have specific side effects such as mouth sores, ulcers, or ulnar pain.  
  - **For obesity:**  
    Bupropion is used to reduce inflammation and control ulcers in conditions like obesity, where weight loss is the goal. It also helps with postmenstrual pain.  
    - The user should discuss this medication with a healthcare provider before starting it due to possible side effects or interactions with other medications.
  - **For bursitis:**  
    Bupropion helps relieve pain in conditions like bursitis, which can be caused by repetitive muscle strains.

- **Risks and Warnings:**  
  The user should be aware of potential risks, including mouth sores or ulcers from repeated use. These side effects are common with drugs like Bupropion and may occur with any medication that causes inflammation, such as naltrexone.

**Safety Warnings:**
- **Meningoencephalolytic effect:**  
  Be cautious of the possibility of meningoencephalolytic (brain-damaging) effects in patients taking Bupropion. This is a rare but serious side effect that may occur with any drug, including this one.
- **Side effects:**  
  While the user has no ulcers or mouth sores after quitting smoking and taking naltrexone, they should be aware of potential mild side effects depending on their specific condition.

**Recommendation:**
The patient is eligible for a prescription medication based on these conditions. They should discuss all medications with a licensed healthcare provider to ensure appropriate dosage, price, and frequency.

In [56]:
question = "اي هو ميلجا"
response = ask_question_with_memory(question,k=3)
Markdown(response)

Drug not found in dataset. Using Ollama to answer via internet...



"ميلجا" is a misspelling. You likely meant **"ميلجّا"**. 

**Millia** (also known as **Milium**) are tiny, white-colored bumps on the skin. 

Here's some information about **milium:**

* **Cause:** They form when dead cells in the skin's outer layer (epidermis) become trapped beneath the surface.
* **Appearance:**  They typically appear as a small, whitish bump, and may be difficult to distinguish from other skin blemishes or miliaria. 
* **Treatment:** In most cases, milia can be treated at home by applying gentle exfoliation or using a warm compress. For severe cases or those resistant to treatment, professional help from a dermatologist is recommended.


Let me know if you have any more questions about skin conditions!

In [ ]:
"Bupropion" in seen_drugs

In [ ]:
question = "what is Bupropion"
response = query_ollama_rag(question)
Markdown(response)

In [ ]:
question = "what is Panadol Extra? "
response = query_ollama_rag(question)
Markdown(response)

In [ ]:
!pip install paddleocr
!pip install paddlepaddle

In [ ]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en')

In [ ]:
from google.colab import files
from PIL import Image

In [ ]:
while True:
  user_input=int(input("1-for image \n 2- for text \n 3- to end\n"))
  if user_input==1:
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
    result = ocr.predict(image_path)

    texts = result[0]['rec_texts']
    full_text = ' '.join(texts)

    question = f"what is {full_text}? "
    response = ask_question_with_memory(question,k=3)
    display(Markdown(response))
  elif user_input==2 :
    question=input("How can i help you\n")
    response = ask_question_with_memory(question,k=3)
    display(Markdown(response))
  elif user_input ==3:
    break

In [ ]:
while True:
  user_input=int(input("1-for image \n 2- for text \n 3- to end\n"))
  if user_input==1:
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
    result = ocr.predict(image_path)

    texts = result[0]['rec_texts']
    full_text = ' '.join(texts)

    question = f"what is {full_text}? "
    response = ask_question_with_memory(question,k=3)
    display(Markdown(response))
  elif user_input==2 :
    question=input("How can i help you\n")
    response = ask_question_with_memory(question,k=3)
    display(Markdown(response))
  elif user_input ==3:
    break

In [ ]:
llm.invoke("الصداع النصفى")

In [ ]:
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
    result = ocr.predict(image_path)

    texts = result[0]['rec_texts']
    full_text = ' '.join(texts)

In [ ]:
print(full_text)

In [ ]:
!pip install -U duckduckgo-search